### SIMA
SIMA is a fully featured toolbox including motion correction and cell detection. Here we illustrate how one can integrate FISSA into their workflow.

For more information about the toolbox see: http://www.losonczylab.org/sima/1.3.2/

Imports

In [ ]:
# SIMA
import sima
import sima.segment

# Plotting toolbox
import holoviews as hv
%load_ext holoviews.ipython

# file operations
import glob

# FISSA
import fissa.core as fissa

Setup data

In [ ]:
# Define folder where tiffs are present
folder = '../exampleData/20150529_mini/'

# Find tiffs in folder
tiffs  = sorted(glob.glob(folder+'/*.tif'))

# define SIMA dataset
sequences = [sima.Sequence.create('TIFF', tiff) for tiff in tiffs[:1]]
try:
    dataset = sima.ImagingDataset(sequences, 'example.sima')
except:
    dataset = sima.ImagingDataset.load('example.sima')


Detect cells with sima

In [ ]:
stica_approach = sima.segment.STICA(components=3)
stica_approach.append(sima.segment.SparseROIsFromMasks())
stica_approach.append(sima.segment.SmoothROIBoundaries())
stica_approach.append(sima.segment.MergeOverlapping(threshold=0.5))
rois = dataset.segment(stica_approach, 'auto_ROIs')

Show detected cells

In [ ]:
fig = hv.Overlay()
for i in range(len(rois)):
    fig*=hv.Curve(rois[i].coords[0][:,1::-1])
fig

Use FISSA with SIMA detected cells

FISSA needs either ImageJ ROIs or numpy arrays as inputs for the ROIs. 

SIMA outputs ROIs as numpy arrays, and can be directly read into FISSA.
A given roi is given as

```
rois[i].coords[0][:,:2].shape
```

SIMA expects a set of rois to be given as a list of lists

```
[[roiA1,roiA2,roiA3...]]
```
So some formatting will need to be done first.

In [ ]:
numROI = len(rois)
rois_FISSA = [rois[i].coords[0][:,:2] for i in range(numROI)]

Which can then be put into FISSA and run

In [ ]:
exp = fissa.Experiment(folder,[rois_FISSA])
exp.separate()

And some plotting to show what it did

In [ ]:
%%opts Curve {+axiswise}
# function for a single cell region plot
def plot_cell_regions(cell):
    out = hv.Overlay()
    numReg = len(exp.roi_polys[cell,0]) # Number of regions
    for i in range(numReg):
        out *= hv.Curve(exp.roi_polys[cell,0][i][0])
    return out

# get plots for all detected regions
region_plots = {i : plot_cell_regions(i) for i in range(exp.nCell)}

# get plots for raw extracts and neuropil removed
traces_plots = {i : hv.Curve(exp.raw[i,1][0,:])*hv.Curve(exp.matched[i,1][0,:]) for i in range(exp.nCell)}

hv.HoloMap(region_plots,kdims=['Cell'])*fig+hv.HoloMap(traces_plots,kdims=['Cell'])